In [62]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, sum
import pandas as pd
import numpy as np

In [63]:
spark = SparkSession.builder.appName("sparkWindowFunctionsExample").getOrCreate()
data = [(1,100,1,2022),(1,700,2,2022),(1,600,2,2022),(2,1000,3,2022),(2,1100,1,2022),(3,400,1,2021),(3,500,1,2021)]
df = spark.createDataFrame(data,schema=["id","salary","month","year"])
df.createOrReplaceTempView("df")
df.show()

+---+------+-----+----+
| id|salary|month|year|
+---+------+-----+----+
|  1|   100|    1|2022|
|  1|   700|    2|2022|
|  1|   600|    2|2022|
|  2|  1000|    3|2022|
|  2|  1100|    1|2022|
|  3|   400|    1|2021|
|  3|   500|    1|2021|
+---+------+-----+----+



In [65]:
window_1=Window.partitionBy(["year","month"]).orderBy("salary")
df=df.withColumn("ytd_sum",sum("salary").over(window_1)).drop("id")
df.show()

+------+-----+----+-------+
|salary|month|year|ytd_sum|
+------+-----+----+-------+
|   400|    1|2021|    400|
|   500|    1|2021|    900|
|   100|    1|2022|    100|
|  1100|    1|2022|   1200|
|   600|    2|2022|    600|
|   700|    2|2022|   1300|
|  1000|    3|2022|   1000|
+------+-----+----+-------+



In [66]:
df2=spark.sql("""SELECT
    salary,
    month,
    year,
    SUM(salary) OVER (PARTITION BY year,month ORDER BY salary) AS ytd_sum
FROM df
ORDER BY year, month;""")
df2.show()

+------+-----+----+-------+
|salary|month|year|ytd_sum|
+------+-----+----+-------+
|   400|    1|2021|    400|
|   500|    1|2021|    900|
|   100|    1|2022|    100|
|  1100|    1|2022|   1200|
|   600|    2|2022|    600|
|   700|    2|2022|   1300|
|  1000|    3|2022|   1000|
+------+-----+----+-------+

